In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool
import pandas as pd
import os
from itertools import chain
import concurrent.futures
import glob
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
# Getting link of a single page
def article_links(soupp):
  
  temp=[]

  tempo = list(soupp.find_all('div',class_=['span-4','post-img-wrap']))

  for i in tempo:
    temp.append(i.find('a').get('href'))

  return temp

# Getting links of multiple pages
def loop_over_pages(date_link):
    links = []
    while True:

        source = requests.get(str(date_link)).text
        soup = BeautifulSoup(source, 'html.parser')
        links.append(article_links(soup))
        if soup.find('a',{'class':'next page-numbers'}) != None:
            date_link = soup.find('a',{'class':'next page-numbers'}).get('href')
#             print("hello next page")
        else:
            break
    return links


In [ ]:
# Getting all links of month of April of 2022
Links  = loop_over_pages("https://www.english.onlinekhabar.com/2022/04")

In [ ]:
def data_extract(link):
    df= pd.DataFrame(columns = ['catagory','heading','text','date','link'])
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    try:
        source = session.get(link,allow_redirects=False)
        soup = BeautifulSoup(source.content.decode("UTF-8"), 'html.parser')
    except:
        return ['None']
    
    #News texts
    text_extracted = ''
    try:
        text_extracted = ''
        article = soup.find("div", {"class":["post-content-wrap"]}).findAll('p')
        for element in article[1:]:
            text_extracted += '\n' + ''.join(element.findAll(text = True))
    except:
        text_extracted = "None"
      
    a=[text_extracted]
    return a

In [ ]:
final_df= pd.DataFrame(columns = ['catagory','heading','text','date','link'])
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    result=executor.map(data_extract,Links)    
for r in result:
    final_df=final_df.append({'text':r[0],ignore_index=True) 
final_df.to_csv(r'./onlinekhabar1.csv',index=False,header=True)